# Mathematical Deep Dive: Treasury Trading Optimization

This notebook explores the mathematics behind the convex optimization problem for treasury trading, minimizing funding costs using 2024 U.S. Treasury real yield curve data.

## Problem Statement

We allocate weights $w_i$ to $n$ treasury maturities to minimize funding costs:

$$ C = \sum_{i=1}^{n} (w_i \cdot r_i + f_i) $$

Where:
- $w_i$: Weight for treasury $i$ (decision variable).
- $r_i$: Real yield rate (e.g., 0.0202).
- $f_i$: Transfer fee (0.001).
- $n$: Number of maturities (5).

Since $\sum f_i$ is constant, we minimize:

$$ C' = \sum_{i=1}^{n} w_i r_i $$

## Constraints

1. **Budget**: Full allocation:
$$ \sum_{i=1}^{n} w_i = 1 $$

2. **Haircut**: Minimum allocation:
$$ w_i \geq h_i = 0.02 \quad \forall i $$

3. **Non-negativity**: $w_i \geq 0$ (subsumed by haircut).

## Linear Programming Formulation

$$ \text{Minimize: } \mathbf{r}^T \mathbf{w} + \mathbf{f}^T \mathbf{1} $$
$$ \text{Subject to: } $$
$$ \mathbf{1}^T \mathbf{w} = 1 $$
$$ \mathbf{w} \geq \mathbf{h} $$

Vectors:
- $\mathbf{w} = [w_1, \dots, w_n]^T$
- $\mathbf{r} = [r_1, \dots, r_n]^T$
- $\mathbf{f} = [0.001, \dots, 0.001]^T$
- $\mathbf{h} = [0.02, \dots, 0.02]^T$

In [1]:
import pandas as pd
import numpy as np
import requests

# Fetch data
url = "https://home.treasury.gov/resource-center/data-chart-center/interest-rates/daily-treasury-rates.csv/2024/all?type=daily_treasury_real_yield_curve&field_tdr_date_value=2024&page&_format=csv"
response = requests.get(url)
with open("treasury_2024.csv", "wb") as f:
    f.write(response.content)

data = pd.read_csv("treasury_2024.csv")
rates = data[["Date", "5 YR", "7 YR", "10 YR", "20 YR", "30 YR"]].dropna()
r = rates.iloc[-1, 1:].values.astype(np.float64) / 100
h = np.array([0.02] * 5)
f = np.array([0.001] * 5)
n = len(r)
print(f"Yields: {r}")

Yields: [0.0176 0.0175 0.0174 0.0184 0.0191]


## Feasibility

Check if $\sum h_i \leq 1$:
$$ 5 \cdot 0.02 = 0.1 \leq 1 $$

The problem is feasible.

## Optimality Insight

Since the objective is linear, the optimal solution lies at a vertex. Typically, allocate maximum weight to the lowest $r_i$, with others at $h_i$:

$$ w_1 = 1 - (n-1) \cdot 0.02, \quad w_{i>1} = 0.02 $$

In [2]:
# Theoretical optimal solution
min_idx = np.argmin(r)
w_theory = np.array([0.02] * n)
w_theory[min_idx] = 1 - (n - 1) * 0.02
cost_theory = np.dot(w_theory, r) + f.sum()
print(f"Theoretical Weights: {w_theory}")
print(f"Theoretical Cost: {cost_theory}")

Theoretical Weights: [0.02 0.02 0.92 0.02 0.02]
Theoretical Cost: 0.02246


## KKT Conditions

1. **Stationarity**: $\mathbf{r} + \lambda \mathbf{1} - \boldsymbol{\mu} = 0$
2. **Primal Feasibility**: $\mathbf{1}^T \mathbf{w} = 1$, $\mathbf{w} \geq \mathbf{h}$
3. **Dual Feasibility**: $\boldsymbol{\mu} \geq 0$
4. **Complementary Slackness**: $\mu_i (w_i - h_i) = 0$

In [3]:
# Example with CVXOPT (open-source)
from cvxopt import matrix, solvers
P = matrix(0.0, (n, n))
q = matrix(r + f)
G = matrix(np.vstack([-np.eye(n), np.ones((1, n))]))
h_cvx = matrix(np.hstack([-h, [1]]))
A = matrix(1.0, (1, n))
b = matrix(1.0)
sol = solvers.qp(P, q, G, h_cvx, A, b)
w_cvxopt = np.array(sol["x"]).flatten()
cost_cvxopt = r @ w_cvxopt + f.sum()
print(f"CVXOPT Weights: {w_cvxopt}")
print(f"CVXOPT Cost: {cost_cvxopt}")

     pcost       dcost       gap    pres   dres
 0:  1.8998e-02 -8.8154e-01  7e+00  2e+00  5e-16
 1:  1.8999e-02 -6.3141e-01  7e-01  2e-02  9e-16
 2:  1.8998e-02  1.0646e-02  8e-03  2e-04  9e-16
 3:  1.8926e-02  1.8382e-02  6e-04  2e-05  7e-14
 4:  1.8485e-02  1.8354e-02  1e-04  1e-06  2e-11
 5:  1.8465e-02  1.8459e-02  6e-06  6e-08  1e-10
 6:  1.8460e-02  1.8460e-02  6e-08  6e-10  6e-10
Optimal solution found.
CVXOPT Weights: [0.02003756 0.02034721 0.91961141 0.02000103 0.0200028 ]
CVXOPT Cost: 0.022460048021173722


## Conclusion

The LP’s linearity ensures solvers converge to the same vertex solution, efficiently balancing low yields with constraints.